In [1]:
# Import Python modules
import os, sys
import yaml
from pathlib import Path
import numpy as np
import numpy.ma as ma
import pandas as  pd
import xarray as xr
from sklearn.cluster import KMeans
from itertools import cycle
# matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.colors import ListedColormap
from matplotlib import rcParams
import matplotlib.ticker as mticker
# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

from IPython.display import Image, display
from scipy import stats

# Path to modules
sys.path.append('../modules')

# Import my modules
from plotter import draw_basemap
from timeseries import persistence, select_months
from eofs import *
from ar_funcs import preprocess_ar_area_subregions
from kmeans import *
import nclcmaps as ncl

In [3]:
# Set up paths

path_to_data = '/home/nash/DATA/data/'                            # project data -- read only
path_to_out  = '/home/nash/DATA/repositories/AR_types/out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '/home/nash/DATA/repositories/AR_types/figs/'      # figures


In [4]:
## Select MERRA2 or ERA5
reanalysis = 'era5'

if reanalysis == 'era5':
    start_date = '1979-01-01'
    end_date = '2019-12-31'
#     filename = 'ar_catalog_ERAI_fraction_HASIAsubregions.nc'
    filename = 'ar_catalog_v3_ERAI_fraction_HASIAsubregions.nc'
## if MERRA2
else:
    start_date = '1980-01-01'
    end_date = '2017-12-31'
    filename = 'ar_catalog_fraction_HASIAsubregions.nc'
    
f1 = path_to_data + 'CH1_generated_data/' + filename
ds = xr.open_dataset(f1)
# Set dates
ds = ds.sel(time=slice(start_date, end_date))
## Preprocess AR subregions - get dataframe of AR days based on area threshold
df = preprocess_ar_area_subregions(df=ds.to_dataframe(), thres=0.3)
# Show table
df.head()

R01  R02  R03  track_id  ar location
time                                                 
1979-01-01  0.000000  0.0  0.0       NaN   0      NaN
1979-01-02  0.000000  0.0  0.0       NaN   0      NaN
1979-01-03  0.005102  0.0  0.0      14.0   0      NaN
1979-01-04  0.000000  0.0  0.0       NaN   0      NaN
1979-01-05  0.000000  0.0  0.0       NaN   0      NaN

In [5]:
# select only rows with AR covering > 30% of area
idx = (df.ar > 0)
df_tmp = df.loc[idx]

djf_df = select_months(df_tmp, 12, 2)
mam_df = select_months(df_tmp, 3, 5)

print('Number of unique events based on tracking algorithm: ')
print('DJF: ', djf_df['track_id'].nunique())
print('MAM: ', mam_df['track_id'].nunique())

Number of unique events based on tracking algorithm: 
DJF:  221
MAM:  280


In [6]:
# how many ARs in HMA between start date and end date?
print('Number of ARs between 1979 and 2019: ', df.ar.sum())
# how many ARs on average per year?
print('Average number of ARs per year: ', df.ar.sum()/40.)

Number of ARs between 1979 and 2019:  1417
Average number of ARs per year:  35.425


In [7]:
def persist_and_duration_stats(x):
    '''
    Count number of independent AR events and their duration in days
    '''
    event_id, nevents, duration = persistence(x.ar)
    print('Total number of AR days in season: ', x.ar.sum())
    print('Total number of independent AR events: ', nevents)
    duration_freq = np.array(np.unique(duration, return_counts=True))
    print(duration_freq)
    bins = np.array([[1, 1], [2, 2], [3, 4], [5, 10]])
    sizes = []
    for i, b in enumerate(bins):
        idx = np.where((duration_freq[0] >= b[0]) & (duration_freq[0] <= b[1]))
        sizes.append((duration_freq[1][idx].sum()/nevents)*100)
    print(sizes)
    
    return sizes

In [8]:
# calculate the number of independent events and event duration using the persistence algorithm
djf_df = select_months(df, 12, 2)
mam_df = select_months(df, 3, 5)
djfmam_df = select_months(df, 12, 5)
djf_sizes = persist_and_duration_stats(djf_df)
mam_sizes = persist_and_duration_stats(mam_df)

sizes = [djf_sizes, mam_sizes]

Total number of AR days in season:  372
Total number of independent AR events:  205
[[ 1.  2.  3.  4.  5.  7.]
 [98. 70. 21. 11.  4.  1.]]
[47.80487804878049, 34.146341463414636, 15.609756097560975, 2.4390243902439024]
Total number of AR days in season:  400
Total number of independent AR events:  237
[[  1.   2.   3.   4.   5.   6.   8.]
 [137.  62.  24.   8.   3.   2.   1.]]
[57.80590717299579, 26.160337552742618, 13.502109704641349, 2.5316455696202533]


In [10]:
fname = path_to_figs + 'donut_plot_duration_freq'
fmt = 'png'

# Pie chart
plt_labels = ['DJF', 'MAM']
labels = ['1 day', '2 days', '3-4 days', '>4 days']
#colors
colors = ['#003660','#0BA89A','#DCE1E5', '#FEBC11']
 
fig = plt.figure()
fig.set_size_inches((8.,4.))
fig.dpi = 300
nrows = 1
ncols = 2
nplots = nrows*ncols
for i in np.arange(nplots):
    ax = plt.subplot(nrows, ncols, i+1)
    ax.pie(sizes[i], colors = colors, labels=labels, autopct=None, startangle=90)
    #draw circle
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    ax.add_artist(centre_circle)
    # Equal aspect ratio ensures that pie is drawn as a circle
    ax.axis('equal')  
    plt.title(plt_labels[i], fontsize=13)

plt.tight_layout()
# plt.show()

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=300)

plotFile = fname + '.png'
print(plotFile)

/home/nash/DATA/repositories/AR_types/figs/donut_plot_duration_freq.png


### Westerly Distubance Comparison

In [ ]:
def resample_track_id(df):
    '''
    Returns an array that has a single AR track ID for each 24 hr
    
    '''
    df_tmp = wd_df['event_id']

    ## resample to 1D taking maximum of new column
    level_values = df_tmp.index.get_level_values
    df_tmp = df_tmp.groupby([level_values(i) for i in [0,1]]+[pd.Grouper(freq='1D', level=0)]).max()


    # date array with all days
    dates_allDays = pd.date_range(start='1979-01-01', end='2015-01-31', freq='1D')
    arr_allDays = np.zeros(len(dates_allDays), dtype=np.float)
    arr_allDays[:] = np.nan

    # Loop over AR days ID and match to list of ALL days 
    for i, date in enumerate(df_tmp.index.get_level_values(2)):
        idx = np.where(dates_allDays == date)
        arr_allDays[idx] = df_tmp.values[i]

    return arr_allDays

In [17]:
filename = path_to_data + 'CEDA/bitmap_era-interim-derived-wd-tracks.csv'
df2 = pd.read_csv(filename, header=29, skiprows=[29], parse_dates=['time'], skipfooter=1, engine='python')
df2 = df2.set_index(pd.to_datetime(df2.time))
df2

event_id         lon        lat     intensity      size  \
time                                                                           
1979-01-01 00:00:00        45   40.290852  37.471957  14426.318010  3.195122   
1979-01-01 00:00:00        48   61.892062  31.935886  19019.216980  3.271355   
1979-01-01 06:00:00        45   43.736661  36.014512  16910.263330  3.201646   
1979-01-01 06:00:00        48   64.076665  31.806202  17289.631140  3.286019   
1979-01-01 12:00:00        45   47.757141  34.824067  16786.617450  3.226879   
...                       ...         ...        ...           ...       ...   
2015-01-31 12:00:00    379773   96.866678  44.752343  12823.753090  3.351098   
2015-01-31 12:00:00    379850   99.100541  29.104065   9946.433741  3.278261   
2015-01-31 18:00:00    379761   61.183830  36.640286  14908.012550  3.204625   
2015-01-31 18:00:00    379773   99.086530  45.359922  13617.449270  3.310938   
2015-01-31 18:00:00    379850  103.364609  29.761811  11065.893700  3.269734   

                     eccentricity      signal                time  vorticity  \
time                                                                           
1979-01-01 00:00:00      0.235435  315.125969 1979-01-01 00:00:00   0.000067   
1979-01-01 00:00:00      0.064982  372.196027 1979-01-01 00:00:00   0.000080   
1979-01-01 06:00:00      0.154904  358.922838 1979-01-01 06:00:00   0.000074   
1979-01-01 06:00:00      0.039174  336.206203 1979-01-01 06:00:00   0.000073   
1979-01-01 12:00:00      0.158068  346.145316 1979-01-01 12:00:00   0.000071   
...                           ...         ...                 ...        ...   
2015-01-31 12:00:00      0.067031  229.623205 2015-01-31 12:00:00   0.000055   
2015-01-31 12:00:00      0.240604  192.133702 2015-01-31 12:00:00   0.000049   
2015-01-31 18:00:00      0.165131  316.832231 2015-01-31 18:00:00   0.000077   
2015-01-31 18:00:00      0.095474  255.577999 2015-01-31 18:00:00   0.000057   
2015-01-31 18:00:00      0.175989  215.445917 2015-01-31 18:00:00   0.000053   

                     timestep_id  
time                              
1979-01-01 00:00:00            0  
1979-01-01 00:00:00            0  
1979-01-01 06:00:00            1  
1979-01-01 06:00:00            1  
1979-01-01 12:00:00            2  
...                          ...  
2015-01-31 12:00:00        52718  
2015-01-31 12:00:00        52718  
2015-01-31 18:00:00        52719  
2015-01-31 18:00:00        52719  
2015-01-31 18:00:00        52719  

[75192 rows x 10 columns]

In [18]:
# total number of WDs 
nevents2 = df2['event_id'].nunique()
print('Total number of events: ', nevents)

Total number of events:  221


In [19]:
wd_df = select_months(df2, 12, 5)
print('Total number of events in DJFMAM: ', wd_df['event_id'].nunique())


Total number of events in DJFMAM:  2631


In [23]:
df_tmp = wd_df['event_id']
## resample to 1D taking maximum of event_id
df_tmp = df_tmp.groupby([pd.Grouper(freq='1D', level=0)]).max()
wd_daily = df_tmp


In [28]:
# select only rows with WWD
idx = (wd_daily.values > 0)
wd_events = wd_daily.loc[idx]

# select only rows with AR
idx = (djfmam_df.ar > 0)
djfmam_df = djfmam_df.loc[idx]

/home/sbarc/students/nash/anaconda3/envs/ar_types/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


In [36]:
x = wd_events.index
y = djfmam_df.index

WD_count = wd_events.loc[x.isin(y)]
perc1 = (len(WD_count)/len(wd_events))*100
print(perc1, '% of WWDs are associated with an AR')
      
AR_count = djfmam_df.loc[y.isin(x)]
perc2 = (len(AR_count)/len(djfmam_df))*100
print(perc2, '% of ARs are associated with a WWD')

10.513005780346822 % of WWDs are associated with an AR
75.38860103626943 % of ARs are associated with a WWD
